In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import json
import requests
import warnings
warnings.filterwarnings('ignore')

# 1. 데이터 로드
population_file = '2025_서울시_구별_인구.csv'
crime_file = '서울시_25개구_5대범죄_연도별통합.csv'

try:
    pop_df = pd.read_csv(population_file, encoding='cp949')
except:
    try:
        pop_df = pd.read_csv(population_file, encoding='euc-kr')
    except:
        pop_df = pd.read_csv(population_file, encoding='utf-8-sig')

try:
    crime_df = pd.read_csv(crime_file, encoding='cp949')
except:
    try:
        crime_df = pd.read_csv(crime_file, encoding='euc-kr')
    except:
        crime_df = pd.read_csv(crime_file, encoding='utf-8-sig')

# 2. 인구 데이터 전처리
if pop_df.iloc[0]['동별(2)'] in ['세대 (세대)', '세대', '계 (명)', '계']:
    pop_df = pop_df.iloc[1:].reset_index(drop=True)

years = [2018, 2019, 2020, 2021, 2022, 2023, 2024]
population_data = []

for idx, row in pop_df.iterrows():
    region = row['동별(2)']

    if pd.isna(region) or region == '' or '총계' in str(region) or '소계' in str(region):
        continue

    for year in years:
        year_cols = [col for col in pop_df.columns if str(year) in str(col)]

        if len(year_cols) >= 2:
            col_name = year_cols[1]

            if pd.notna(row[col_name]):
                pop_value = row[col_name]
                if isinstance(pop_value, str):
                    pop_value = pop_value.replace(',', '').strip()
                try:
                    pop_value = float(pop_value)
                    population_data.append({
                        'region': region,
                        'year': year,
                        'population': pop_value
                    })
                except:
                    pass

pop_clean = pd.DataFrame(population_data)

# 3. 범죄율 계산
merged_df = pd.merge(crime_df, pop_clean, on=['region', 'year'], how='inner')
merged_df['crime_rate'] = (merged_df['total'] / merged_df['population']) * 100000

# 호버 텍스트 생성
merged_df['text'] = merged_df.apply(
    lambda x: f"<b>{x['region']}</b><br>범죄율: {x['crime_rate']:.1f}<br>범죄 건수: {x['total']:,}건<br>인구: {x['population']:,.0f}명",
    axis=1
)

# 4. 서울시 GeoJSON 다운로드
geojson_url = 'https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json'

try:
    response = requests.get(geojson_url)
    seoul_geo = response.json()

    # GeoJSON에서 구별 중심 좌표 추출
    centroids = {}
    for feature in seoul_geo['features']:
        name = feature['properties']['name']
        coords = feature['geometry']['coordinates'][0]

        if feature['geometry']['type'] == 'MultiPolygon':
            coords = feature['geometry']['coordinates'][0][0]

        lats = [coord[1] for coord in coords]
        lons = [coord[0] for coord in coords]
        centroids[name] = {
            'lat': sum(lats) / len(lats),
            'lon': sum(lons) / len(lons)
        }

    merged_df['lat'] = merged_df['region'].map(lambda x: centroids.get(x, {}).get('lat'))
    merged_df['lon'] = merged_df['region'].map(lambda x: centroids.get(x, {}).get('lon'))

except Exception as e:
    print(f" 오류: GeoJSON 로드 실패 - {e}")
    seoul_geo = None

# 5. Plotly Choropleth 지도 생성
if seoul_geo:
    # 색상 범위를 분위수 기준으로 조정
    all_rates = merged_df['crime_rate'].values
    q25 = np.percentile(all_rates, 25)
    q75 = np.percentile(all_rates, 75)

    fig = go.Figure()

    # 각 연도별 프레임 생성
    for year in sorted(merged_df['year'].unique()):
        year_data = merged_df[merged_df['year'] == year].copy()

        # Choropleth 레이어
        fig.add_trace(go.Choroplethmapbox(
            geojson=seoul_geo,
            locations=year_data['region'],
            z=year_data['crime_rate'],
            featureidkey="properties.name",
            colorscale='YlOrRd',
            zmin=q25,
            zmax=q75,
            marker_opacity=0.7,
            marker_line_width=1,
            marker_line_color='white',
            text=year_data['text'],
            hovertemplate='%{text}<extra></extra>',
            colorbar=dict(
                title="범죄율<br>(10만명당)",
                thickness=15,
                len=0.7,
                x=1.02
            ),
            name=str(year),
            visible=(year == 2018)
        ))

        # 각 구에 텍스트 라벨 추가 (지역명만 표시)
        fig.add_trace(go.Scattermapbox(
            lat=year_data['lat'],
            lon=year_data['lon'],
            mode='text',
            text=year_data['region'],
            textfont=dict(
                size=11,
                color='black',
                family='Arial Black'
            ),
            hoverinfo='skip',
            name=f'{year}_labels',
            visible=(year == 2018)
        ))

    # 슬라이더 생성
    steps = []
    for i, year in enumerate(sorted(merged_df['year'].unique())):
        step = dict(
            method="update",
            args=[
                {"visible": [False] * len(fig.data)},
                {"title": f"서울시 구별 범죄율 ({year}년)"}
            ],
            label=str(year)
        )
        step["args"][0]["visible"][i*2] = True
        step["args"][0]["visible"][i*2 + 1] = True
        steps.append(step)

    sliders = [dict(
        active=0,
        yanchor="top",
        y=0.02,
        xanchor="left",
        x=0.05,
        currentvalue=dict(
            prefix="년도: ",
            visible=True,
            xanchor="left",
            font=dict(size=16, color='#333')
        ),
        pad=dict(b=10, t=10),
        len=0.9,
        steps=steps
    )]

    # 레이아웃 설정
    fig.update_layout(
        mapbox=dict(
            style="open-street-map",
            center=dict(lat=37.5665, lon=126.9780),
            zoom=10
        ),
        sliders=sliders,
        title=dict(
            text='서울시 구별 범죄율 (2018년)',
            font=dict(size=24, color='#333'),
            x=0.5,
            xanchor='center'
        ),
        height=700,
        margin=dict(l=0, r=0, t=50, b=0),
        showlegend=False,
        # 확대/축소 및 이동 가능하게 설정
        dragmode='zoom'
    )

    # 확대/축소 버튼 설정
    fig.update_layout(
        mapbox=dict(
            style="open-street-map",
            center=dict(lat=37.5665, lon=126.9780),
            zoom=10
        ),
        # 줌 컨트롤 활성화
        updatemenus=[],
    )

    fig.show()

else:
    print(" 오류: GeoJSON을 로드할 수 없습니다.")

In [7]:
import pandas as pd
import numpy as np

# 1. 데이터 로드
population_file = '2025_서울시_구별_인구.csv'
crime_file = '서울시_25개구_5대범죄_연도별통합.csv'
pop_df = pd.read_csv(population_file, encoding='cp949')

# 인코딩을 시도하며 읽기
try:
    pop_df = pd.read_csv(population_file, encoding='cp949')
except:
    try:
        pop_df = pd.read_csv(population_file, encoding='euc-kr')
    except:
        pop_df = pd.read_csv(population_file, encoding='utf-8-sig')

try:
    crime_df = pd.read_csv(crime_file, encoding='cp949')
except:
    try:
        crime_df = pd.read_csv(crime_file, encoding='euc-kr')
    except:
        crime_df = pd.read_csv(crime_file, encoding='utf-8-sig')

print("✅ 데이터 로드 완료")

# 2. 인구 데이터 전처리
if pop_df.iloc[0]['동별(2)'] in ['세대 (세대)', '세대', '계 (명)', '계']:
    pop_df = pop_df.iloc[1:].reset_index(drop=True)

years = [2025, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
population_data = []

for idx, row in pop_df.iterrows():
    region = row['동별(2)']

    if pd.isna(region) or region == '' or '총계' in str(region) or '소계' in str(region):
        continue

    for year in years:
        year_cols = [col for col in pop_df.columns if str(year) in str(col)]

        if len(year_cols) >= 2:
            col_name = year_cols[1]

            if pd.notna(row[col_name]):
                pop_value = row[col_name]
                if isinstance(pop_value, str):
                    pop_value = pop_value.replace(',', '').strip()
                try:
                    pop_value = float(pop_value)
                    population_data.append({
                        'region': region,
                        'year': year,
                        'population': pop_value
                    })
                except:
                    pass

pop_clean = pd.DataFrame(population_data)
print(f" 인구 데이터 전처리 완료: {len(pop_clean)}개 레코드")

# 3. 범죄율 계산
merged_df = pd.merge(crime_df, pop_clean, on=['region', 'year'], how='inner')
merged_df['crime_rate'] = (merged_df['total'] / merged_df['population']) * 100000
merged_df['crime_rate'] = merged_df['crime_rate'].round(2)

print(f" 범죄율 계산 완료: {len(merged_df)}개 레코드")

# 4. 데이터 정렬 및 저장
merged_df_sorted = merged_df.sort_values(['year', 'region']).reset_index(drop=True)

output_file = 'crime_rate_long.csv'
merged_df_sorted[['region', 'year', 'population', 'total', 'crime_rate']].to_csv(
    output_file,
    index=False,
    encoding='utf-8-sig'
)

print(f"\n CSV 저장 완료: {output_file}")
print("   형식: region, year, population, total, crime_rate")

# 5. 미리보기
print("\n" + "="*60)
print("📄 저장된 데이터 미리보기")
print("="*60)
print(merged_df_sorted[['region', 'year', 'crime_rate']].head(10))

print(f"\n✨ {output_file} 파일이 생성되었습니다.")

✅ 데이터 로드 완료
✅ 인구 데이터 전처리 완료: 200개 레코드
✅ 범죄율 계산 완료: 175개 레코드

✅ CSV 저장 완료: crime_rate_long.csv
   형식: region, year, population, total, crime_rate

📄 저장된 데이터 미리보기
  region  year  crime_rate
0    강남구  2018     1372.36
1    강동구  2018      907.34
2    강북구  2018     1064.37
3    강서구  2018      766.88
4    관악구  2018      966.46
5    광진구  2018     1055.08
6    구로구  2018     1096.96
7    금천구  2018     1296.35
8    노원구  2018      730.99
9    도봉구  2018      658.28

✨ crime_rate_long.csv 파일이 생성되었습니다.
